In [102]:
import numpy as np
import pandas as pd
import requests
import json

In [110]:
import arrow
import dateutil
from dateutil.parser import parse
import datetime as dt
import pytz

getting weather *measurements* datapoints (for solar irradiance) from NREL SRRL BMS through the API:<br>
https://internal-apis.nrel.gov/intelligentcampus/hisRead?id=@p:nrel:r:20ed5df2-2c0e126b&range="2018-08-31%2c2018-09-02"

In the above request, note the manner in which the date range is specified.

to plug in the ID of the specific measurement (Dry Bulb Temperature in the following example) in the above link:<br>
NREL => Historian => Weather => NREL SRRL BMS => Dry Bulb Temperature => Aspects => Folio => ID

List of ids:
    - Energy Consumption => unit kwh => @p:nrel:r:225918db-bfbda16a
    - Relative Humidity (RH) => unit %RH  => @p:nrel:r:20ed5e0a-275dbdc2
    - Barometric Pressure (BP) => unit mbar  => @p:nrel:r:20ed5e0a-53e174aa
    - Dry Bulb Temperature (DBT) => unit degree Far => @p:nrel:r:20ed5e0a-fe755c80
    - Global Horizontal Irradiance (GHI) => unit watss/m2_irr => @p:nrel:r:20ed5df2-2c0e126b
    - Total Cloud Cover(TCC) => unit % => @p:nrel:r:20ed5e0a-acc8beff
    - Wind Speed at 19’ (5.7 meter) (WS) => unit mph => @p:nrel:r:20ed5df2-fd2eecc5
    

In [75]:
resp1 = requests.get("https://internal-apis.nrel.gov/intelligentcampus/hisRead?id=@p:nrel:r:225918db-bfbda16a&range=\"2018-08-31%2c2018-09-02\"")
if resp1.status_code == 200:
    pass
else:
    print("Energy consumption data not coming from API")

EC = resp1.content.decode('utf-8').split("\n")
EC = EC[2:]

In [112]:
EC[25]

'2018-08-31T06:15:00-06:00 Denver,1051.4326171875kWh'

In [117]:
dt = parse(EC[25].split(",")[0].split(" ")[0])
print(dt.date())
print(dt.time())
print(dt.tzinfo)

2018-08-31
06:15:00
tzoffset(None, -21600)
